# Advanced SQL for Data Scientists

## Chapter 1: Overview

### Data Management Operations
* Linking data from different data stores
* Filtering and reformatting data for different uses
* Aggregating data to provide 'big picture' summaries
* Answering specific questions about business operations

### Data Sources
* Relational Databases
* NoSQL Database:
    * Non-relational databases: NoSQL
* 